In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
from nest_asyncio import apply
apply()

In [4]:
from chat import *
import os

In [5]:
from psql_helpers import (
    async_session, get_session,

)
from vexa import *

In [6]:
auth = VexaAuth()
vexa = VexaAPI(token="")
user_id = (await vexa.get_user_info())['id']

# Initialize search engines

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [7]:
qdrant_engine = QdrantSearchEngine(os.getenv('VOYAGE_API_KEY'))
es_engine = await ElasticsearchBM25.create() 

Connecting to Elasticsearch at elasticsearch:9200


In [9]:
chat_manager = UnifiedChatManager(
        session=get_session(),
        qdrant_engine=qdrant_engine,
        es_engine=es_engine
    )

In [10]:
query = "What is the weather in San Francisco?"

In [11]:
fronten_id = None
thread_id = None
content_id = None
entity_id = None
content_ids = None
entity_ids = None
model = "gpt-4o-mini"
temperature = 0.0
meta = None


In [12]:
from helpers import jupyter_stream_output


In [13]:
from psql_helpers import get_session

@jupyter_stream_output(markdown=True)
async def unified_chat(
    query: str,
    content_id: Optional[UUID] = None,
    entity_id: Optional[int] = None,
    content_ids: Optional[List[UUID]] = None,
    entity_ids: Optional[List[int]] = None,
    thread_id: Optional[str] = None,
    temperature: float = 0.7
):
    """
    Unified chat function supporting both content and entity-based search
    
    Args:
        query: The user's query
        content_id: Optional single content ID for direct thread mapping
        entity_id: Optional single entity ID for direct thread mapping
        content_ids: Optional list of content IDs for search scope
        entity_ids: Optional list of entity IDs for search scope
        thread_id: Optional thread ID to continue conversation
        temperature: Model temperature parameter
    """
    
    async with get_session() as session:
        chat_manager = UnifiedChatManager(
            session=session,
            qdrant_engine=qdrant_engine,
            es_engine=es_engine
        )
        
        async for result in chat_manager.chat(
            user_id=user_id,
            query=query,
            content_id=content_id,
            entity_id=entity_id,
            content_ids=content_ids,
            entity_ids=entity_ids,
            thread_id=thread_id,
            temperature=temperature
        ):
            if 'chunk' in result:
                yield result['chunk']
            elif 'linked_output' in result:
                yield result
                
            print(result)

In [14]:
await unified_chat(query="what is my name?")



Your name is **Dmitriy Grankin**.

{'chunk': '.'}


{'thread_id': 'dcbae2f5-a34e-423b-85b8-b832cf3cea1c',
 'output': 'Your name is **Dmitriy Grankin**.',
 'linked_output': 'Your name is **Dmitriy Grankin**.',
 'service_content': {'output': 'Your name is **Dmitriy Grankin**.',
  'context': 'No relevant context found.'}}

In [28]:
await unified_chat(query="who is this guy?",entity_id=5432)



It seems there isn't enough context to identify "this guy." If you could provide more details or specify who you are referring to, I would be happy to help!

{'chunk': '!'}


{'thread_id': 'ef7f61aa-4602-448b-aeed-622b47a741d0',
 'output': 'It seems there isn\'t enough context to identify "this guy." If you could provide more details or specify who you are referring to, I would be happy to help!',
 'linked_output': 'It seems there isn\'t enough context to identify "this guy." If you could provide more details or specify who you are referring to, I would be happy to help!',
 'service_content': {'output': 'It seems there isn\'t enough context to identify "this guy." If you could provide more details or specify who you are referring to, I would be happy to help!',
  'context': 'No relevant context found.'}}